In [189]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
# import tensorflow.keras.models 
from keras.models import model_from_json
import pickle

In [190]:
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5



# Load model and labels

In [191]:
json_file = open('/home/shivasankaran/saved_models/eqsolver/eqnsolver_v1.json', 'r')
my_model_json = json_file.read()
json_file.close()
my_model = model_from_json(my_model_json)
# load weights into new model
my_model.load_weights("/home/shivasankaran/saved_models/eqsolver/eqnsolver_v1.h5")
print(my_model.summary())

l_file = open("/home/shivasankaran/datasets/equation solver/labels.pkl", "rb")
label = pickle.load(l_file)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 30, 24, 24)        780       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 12, 12)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 10, 10)        4065      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 5, 5)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 5, 5)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 375)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)              

# Feature detector

In [192]:
def rectContains(rect,pt):
    if(rect[0] < pt[0] and pt[0]<rect[0]+rect[2] and rect[1] < pt[1] and pt[1]< rect[1]+rect[3]):
        return True
    else:
        return False


In [193]:
def feature_extractor(image_path):
    features = []
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
#     kernel = np.ones((3,3), np.uint8)
#     kernel2 = np.ones((4,4),np.uint8)
    thresh,binary = cv2.threshold(gray,130,255,cv2.THRESH_BINARY)
#     binary = cv2.dilate(binary, kernel, iterations=1)
#     binary = cv2.GaussianBlur(binary,(3,3),cv2.BORDER_DEFAULT)
#     binary = cv2.erode(binary, kernel2, iterations=1)
#     cv2.medianBlur(binary, 21)
#     cv2.fastNlMeansDenoising(binary,binary)
    contours,_ = cv2.findContours(binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    with_contours = cv2.drawContours(binary, contours, -1,(255,0,255),3)
    
    rects = []
    for contour in contours:
        if(cv2.contourArea(contour)>400 and cv2.contourArea(contour)<50000):
            x,y,w,h = cv2.boundingRect(contour)
            rect = [x,y,w,h]
            rects.append(rect)

    rects.sort(key = lambda k:k[0])
        
    temp = []
    for i in range(len(rects)):
        r = rects[i]
        l=[]
        for j in range(i+1,len(rects)):
            rec = rects[j]
            x=rec[0]
            y=rec[1]
            w=rec[2]
            h=rec[3]
            top_left = [x,y]
            bottom_right = [x+w,y+h]
            if(rectContains(r,top_left) or rectContains(r,bottom_right) ):
                if(w*h>r[2]*r[3]):
                    temp.append(i)
                else:
                    temp.append(j)
    final_rects = [] 
    for i in range(len(rects)):
        if i not in temp:
            final_rects.append(rects[i])
    y_ref = 0
    h_ref = 0
    for r in final_rects:
        y_ref+=(r[1]+(r[3])/2)
        h_ref+=r[3]
    y_ref = y_ref//len(final_rects)
    h_ref = h_ref//len(final_rects)
    h_ref = int(h_ref)
    y_ref = int(y_ref)
#     cpy = binary.copy()
#     cv2.line(cpy,(0,y_ref),(1200,y_ref),(0, 255, 0),10)
#     plt.imshow(cpy, cmap = 'binary', interpolation = 'bicubic')
#     plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
#     plt.show()
    for rect in final_rects:
        tp = "element"
        x=rect[0]
        y=rect[1]
        w=rect[2]
        h=rect[3]
        if(y+h<y_ref and h_ref>h):
            tp = "superscript"
        if(y>y_ref and h_ref>h):
            tp = "subscript"
            
            
        boxed = binary[y-5:y+h+5,x-5:x+w+5]
        
        
        resized = cv2.resize(boxed,(28,28))
        features.append([resized,tp])
    return features


    return features
    

# Equation detctor

In [194]:
data_test_image = "/home/shivasankaran/datasets/equation solver/data/extracted_images/3/3_108.jpg"
operators = ["+","-","/","*"]
my_test = "/home/shivasankaran/datasets/equation solver/test9.jpeg"
feats = feature_extractor(my_test)
s = ""
for feat,tp in feats:
#     plt.imshow(feat, cmap = 'binary', interpolation = 'bicubic')
#     plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
#     plt.show()
    
    resized_img = np.reshape(feat,(1,1,28,28))
    predictions = my_model.predict_classes(resized_img)
    predicted = labels[predictions[0]]
    s+=predicted
#     print(predicted)
    if(predicted not in operators):
        if(tp == "subscript"):
            s+="(sub)"
        if(tp == "superscript"):
            s+="(sup)"
    
print("Predicted for the above img:",s)

Predicted for the above img: X3(sup)+y4(sub)3(sup)+4X5(sup)
